# Similar Question finder

In [64]:
import gensim
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SEEKER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [66]:
# preprocess text
def preprocess_data(text):
    STOPWORDS = set(stopwords.words('english'))
    # convert to lowercase
    text = text.lower()
    # replace whitespaces and punctuations
    text = re.sub('[/(){}\[\]\|@,;]', ' ', text)
    text = re.sub('[^0-9a-z #+_]', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

In [21]:
def load_data(filename):
    data = []
    # open the file and load the training questions with positives and
    # negatives
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(line.strip().split('\t'))
    return data

In [22]:
data = load_data(filename='data/validation.tsv')

In [23]:
def rank_duplicate_candidates(question, candidates, embeddings, dim=300):
    # get embeddings for question 
    question_embed = question_embedding(question, word_embeddings, dim=dim)
    # get embeddings for candidates
    candidates_embed = np.array([question_embedding(candidate, word_embeddings) 
                          for candidate in candidates])
    # compute cosine similarity
    candidates_sim = cosine_similarity(question_embed.reshape(1,-1), candidates_embed)[0]
    # make a tuple pair(sim, candidate question index)
    candidates_ques_sim = [(sim, i) for i, sim in enumerate(candidates_sim)]
    # sort the list in descending order
    candidates_ques_sim = sorted(candidates_ques_sim, reverse=True)
    final_candidates_ranking = [(index, candidates[index]) for _,index in candidates_ques_sim]
    
    return final_candidates_ranking

In [24]:
word_embeddings = KeyedVectors.load_word2vec_format(
                    'embeddings/GoogleNews-vectors-negative300.bin',
                    binary=True, limit=500000)

In [4]:
# sanity checking
print('neural' in word_embeddings)
print(word_embeddings.most_similar(positive=['woman', 'king'], negative=['man']))

True
[('queen', 0.7118192911148071), ('monarch', 0.6189674139022827), ('princess', 0.5902431011199951), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321243286133), ('kings', 0.5236844420433044), ('queens', 0.518113374710083), ('sultan', 0.5098593235015869), ('monarchy', 0.5087411999702454), ('royal_palace', 0.5087165832519531)]


In [3]:
word_embeddings['apple'].shape

(300,)

In [25]:
# for representing the question as vector of embeddings for 
# known words in question. We take the mean of all the embeddings for the 
# words of the question
def question_embedding(question, word_embeddings, dim=300):
    # tokenize the question 
    question = question.split()
    
    question_vector = [word_embeddings[word] for word in question if word in word_embeddings]
    
    # return if there is atleast one word known to the pretrained embedding
    if len(question_vector) == 0:
        return np.zeros(dim) 
    
    # take the mean along the axis = 0(along each column).
    # since each embedding is (300,1)
    # so all of them will have shape (no. of words, 300)
    question_vector = np.array(question_vector)
    question_vector = np.mean(question_vector, axis=0)
    
    return question_vector 

## <u>Evaluation metrics
We will use two evaluation metrics for this task.
    
***1. Hits@k***

***2. DCG@k***
    

In [26]:
'''
    ranks: list of ranks for each duplicate
    k: (int) stating the window size to scan
'''
def hits_k(k, ranks):
    hit_score = np.array(ranks) <= k
    hit_score = hit_score.mean()
    return hit_score

In [27]:
'''
    ranks: list of ranks for each duplicate
    k: (int) stating the window size to scan
'''
def dcg_k(k, ranks):
    dcg_score = np.array(ranks) <= k
    dcg_score = dcg_score/np.log2(1 + np.array(ranks))
    dcg_score = dcg_score.mean()
    
    return dcg_score

In [68]:
data_processed = []
for i, line in tqdm(enumerate(data)):
    data_processed.append([])
    for j, l in tqdm(enumerate(line)):
        data_processed[i].append(preprocess_data(l))



0it [00:00, ?it/s]


0it [00:00, ?it/s]


368it [00:00, 3364.13it/s]


786it [00:00, 3592.78it/s]


1001it [00:00, 3768.24it/s]

1it [00:00,  3.56it/s]


0it [00:00, ?it/s]


392it [00:00, 3583.79it/s]


789it [00:00, 3606.60it/s]


1001it [00:00, 3768.23it/s]

2it [00:00,  3.66it/s]


0it [00:00, ?it/s]


394it [00:00, 3602.07it/s]


798it [00:00, 3647.60it/s]


1001it [00:00, 3768.23it/s]

3it [00:00,  3.62it/s]


0it [00:00, ?it/s]


417it [00:00, 3808.58it/s]


827it [00:00, 3778.44it/s]


1001it [00:00, 3766.73it/s]

4it [00:01,  3.61it/s]


0it [00:00, ?it/s]


346it [00:00, 3166.32it/s]


731it [00:00, 3342.97it/s]


1001it [00:00, 3560.24it/s]

5it [00:01,  3.56it/s]


0it [00:00, ?it/s]


382it [00:00, 3492.06it/s]


770it [00:00, 3519.77it/s]


1001it [00:00, 3558.88it/s]

6it [00:01,  3.56it/s]


0it [00:00, ?it/s]


359it [00:00, 3282.09it/s]


761it [00:00, 3478.46it/s]


1001it [00:00, 3558.76it/s]

7it [00:01,  3.56it/s]


0it [00:00, ?it/s]


382it [00:00, 3492.60it/s

61it [00:17,  3.54it/s]


0it [00:00, ?it/s]


361it [00:00, 3300.08it/s]


770it [00:00, 3519.65it/s]


1001it [00:00, 3558.88it/s]

62it [00:17,  3.54it/s]


0it [00:00, ?it/s]


397it [00:00, 3629.16it/s]


810it [00:00, 3702.59it/s]


1001it [00:00, 3768.07it/s]

63it [00:17,  3.55it/s]


0it [00:00, ?it/s]


360it [00:00, 3291.18it/s]


760it [00:00, 3474.21it/s]


1001it [00:00, 3558.90it/s]

64it [00:18,  3.55it/s]


0it [00:00, ?it/s]


373it [00:00, 3410.30it/s]


781it [00:00, 3570.04it/s]


1001it [00:00, 3558.85it/s]

65it [00:18,  3.55it/s]


0it [00:00, ?it/s]


376it [00:00, 3437.80it/s]


772it [00:00, 3528.92it/s]


1001it [00:00, 3558.99it/s]

66it [00:18,  3.55it/s]


0it [00:00, ?it/s]


386it [00:00, 3528.57it/s]


793it [00:00, 3624.76it/s]


1001it [00:00, 3768.22it/s]

67it [00:18,  3.55it/s]


0it [00:00, ?it/s]


411it [00:00, 3757.46it/s]


803it [00:00, 3670.63it/s]


1001it [00:00, 3768.23it/s]

68it [00:19,  3.55it/s]


0it [00:00, ?it/s]


345it [00:00, 3

758it [00:00, 3464.76it/s]


1001it [00:00, 3558.87it/s]

122it [00:34,  3.57it/s]


0it [00:00, ?it/s]


328it [00:00, 2998.65it/s]


697it [00:00, 3186.07it/s]


1001it [00:00, 3371.56it/s]

123it [00:34,  3.57it/s]


0it [00:00, ?it/s]


395it [00:00, 3611.17it/s]


801it [00:00, 3661.46it/s]


1001it [00:00, 3768.07it/s]

124it [00:34,  3.57it/s]


0it [00:00, ?it/s]


383it [00:00, 3501.48it/s]


775it [00:00, 3542.46it/s]


1001it [00:00, 3371.55it/s]

125it [00:35,  3.57it/s]


0it [00:00, ?it/s]


298it [00:00, 2724.38it/s]


681it [00:00, 3112.80it/s]


1001it [00:00, 3202.99it/s]

126it [00:35,  3.56it/s]


0it [00:00, ?it/s]


337it [00:00, 3080.93it/s]


714it [00:00, 3263.64it/s]


1001it [00:00, 3371.56it/s]

127it [00:35,  3.56it/s]


0it [00:00, ?it/s]


378it [00:00, 3455.80it/s]


785it [00:00, 3588.18it/s]


1001it [00:00, 3768.23it/s]

128it [00:35,  3.56it/s]


0it [00:00, ?it/s]


410it [00:00, 3748.35it/s]


812it [00:00, 3711.74it/s]


1001it [00:00, 3768.08it/s

0it [00:00, ?it/s]


384it [00:00, 3510.29it/s]


797it [00:00, 3643.03it/s]


1001it [00:00, 3768.08it/s]

183it [00:51,  3.58it/s]


0it [00:00, ?it/s]


354it [00:00, 3236.34it/s]


739it [00:00, 3378.06it/s]


1001it [00:00, 3558.86it/s]

184it [00:51,  3.58it/s]


0it [00:00, ?it/s]


404it [00:00, 3693.48it/s]


808it [00:00, 3693.48it/s]


1001it [00:00, 3768.09it/s]

185it [00:51,  3.58it/s]


0it [00:00, ?it/s]


374it [00:00, 3419.18it/s]


792it [00:00, 3618.72it/s]


1001it [00:00, 3768.22it/s]

186it [00:52,  3.58it/s]


0it [00:00, ?it/s]


366it [00:00, 3346.12it/s]


788it [00:00, 3602.05it/s]


1001it [00:00, 3559.00it/s]

187it [00:52,  3.58it/s]


0it [00:00, ?it/s]


382it [00:00, 3492.35it/s]


770it [00:00, 3519.60it/s]


1001it [00:00, 3558.75it/s]

188it [00:52,  3.58it/s]


0it [00:00, ?it/s]


379it [00:00, 3465.23it/s]


786it [00:00, 3592.91it/s]


1001it [00:00, 3768.21it/s]

189it [00:52,  3.58it/s]


0it [00:00, ?it/s]


394it [00:00, 3601.72it/s]


795it

1001it [00:00, 3768.09it/s]

243it [01:07,  3.58it/s]


0it [00:00, ?it/s]


384it [00:00, 3510.93it/s]


799it [00:00, 3652.48it/s]


1001it [00:00, 3768.23it/s]

244it [01:08,  3.58it/s]


0it [00:00, ?it/s]


367it [00:00, 3355.19it/s]


768it [00:00, 3510.74it/s]


1001it [00:00, 3558.85it/s]

245it [01:08,  3.58it/s]


0it [00:00, ?it/s]


364it [00:00, 3328.08it/s]


778it [00:00, 3556.34it/s]


1001it [00:00, 3768.22it/s]

246it [01:08,  3.59it/s]


0it [00:00, ?it/s]


419it [00:00, 3830.35it/s]


827it [00:00, 3780.20it/s]


1001it [00:00, 3768.07it/s]

247it [01:08,  3.59it/s]


0it [00:00, ?it/s]


397it [00:00, 3629.48it/s]


797it [00:00, 3643.37it/s]


1001it [00:00, 3768.38it/s]

248it [01:09,  3.59it/s]


0it [00:00, ?it/s]


363it [00:00, 3318.35it/s]


782it [00:00, 3574.46it/s]


1001it [00:00, 3768.07it/s]

249it [01:09,  3.59it/s]


0it [00:00, ?it/s]


419it [00:00, 3830.28it/s]


835it [00:00, 3816.91it/s]


1001it [00:00, 4003.60it/s]

250it [01:09,  3.59it/s]



...

353it [00:00, 3494.25it/s]


707it [00:00, 3516.44it/s]


1001it [00:00, 3546.54it/s]

9732it [47:11,  3.44it/s]


0it [00:00, ?it/s]


357it [00:00, 3512.69it/s]


712it [00:00, 3530.80it/s]


1001it [00:00, 3528.57it/s]

9733it [47:11,  3.44it/s]


0it [00:00, ?it/s]


353it [00:00, 3494.28it/s]


712it [00:00, 3536.37it/s]


1001it [00:00, 3483.32it/s]

9734it [47:12,  3.44it/s]


0it [00:00, ?it/s]


357it [00:00, 3533.90it/s]


708it [00:00, 3521.60it/s]


1001it [00:00, 3523.71it/s]

9735it [47:12,  3.44it/s]


0it [00:00, ?it/s]


359it [00:00, 3553.67it/s]


705it [00:00, 3506.68it/s]


1001it [00:00, 3499.21it/s]

9736it [47:12,  3.44it/s]


0it [00:00, ?it/s]


354it [00:00, 3504.20it/s]


701it [00:00, 3504.06it/s]


1001it [00:00, 3486.93it/s]

9737it [47:13,  3.44it/s]


0it [00:00, ?it/s]


343it [00:00, 3395.30it/s]


696it [00:00, 3461.92it/s]


1001it [00:00, 3450.96it/s]

9738it [47:13,  3.44it/s]


0it [00:00, ?it/s]


357it [00:00, 3569.21it/s]


712it [00:00, 3541.

1001it [00:00, 3548.86it/s]

9791it [47:28,  3.44it/s]


0it [00:00, ?it/s]


345it [00:00, 3415.11it/s]


696it [00:00, 3479.09it/s]


1001it [00:00, 3474.82it/s]

9792it [47:29,  3.44it/s]


0it [00:00, ?it/s]


353it [00:00, 3528.88it/s]


703it [00:00, 3514.03it/s]


1001it [00:00, 3486.90it/s]

9793it [47:29,  3.44it/s]


0it [00:00, ?it/s]


355it [00:00, 3514.04it/s]


708it [00:00, 3519.72it/s]


1001it [00:00, 3499.09it/s]

9794it [47:29,  3.44it/s]


0it [00:00, ?it/s]


349it [00:00, 3454.41it/s]


704it [00:00, 3501.35it/s]


1001it [00:00, 3511.35it/s]

9795it [47:29,  3.44it/s]


0it [00:00, ?it/s]


355it [00:00, 3548.49it/s]


711it [00:00, 3553.86it/s]


1001it [00:00, 3380.87it/s]

9796it [47:30,  3.44it/s]


0it [00:00, ?it/s]


308it [00:00, 3079.06it/s]


608it [00:00, 3024.05it/s]


936it [00:00, 3108.84it/s]


1001it [00:00, 3098.26it/s]

9797it [47:30,  3.44it/s]


0it [00:00, ?it/s]


325it [00:00, 3217.10it/s]


654it [00:00, 3253.01it/s]


991it [00:00, 3281.

1001it [00:00, 2791.45it/s]

9848it [47:46,  3.44it/s]


0it [00:00, ?it/s]


334it [00:00, 3053.26it/s]


728it [00:00, 3327.79it/s]


1001it [00:00, 3558.88it/s]

9849it [47:46,  3.44it/s]


0it [00:00, ?it/s]


391it [00:00, 3574.31it/s]


796it [00:00, 3638.62it/s]


1001it [00:00, 3768.08it/s]

9850it [47:46,  3.44it/s]


0it [00:00, ?it/s]


407it [00:00, 3720.93it/s]


808it [00:00, 3693.29it/s]


1001it [00:00, 3768.24it/s]

9851it [47:46,  3.44it/s]


0it [00:00, ?it/s]


330it [00:00, 3013.75it/s]


727it [00:00, 3323.20it/s]


1001it [00:00, 3558.73it/s]

9852it [47:47,  3.44it/s]


0it [00:00, ?it/s]


395it [00:00, 3611.19it/s]


800it [00:00, 3656.74it/s]


1001it [00:00, 3768.09it/s]

9853it [47:47,  3.44it/s]


0it [00:00, ?it/s]


351it [00:00, 3208.92it/s]


747it [00:00, 3414.64it/s]


1001it [00:00, 3558.87it/s]

9854it [47:47,  3.44it/s]


0it [00:00, ?it/s]


340it [00:00, 3105.76it/s]


736it [00:00, 3364.35it/s]


1001it [00:00, 3558.98it/s]

9855it [47:48,  3.4

356it [00:00, 3524.00it/s]


703it [00:00, 3514.06it/s]


1001it [00:00, 3523.87it/s]

9906it [48:03,  3.44it/s]


0it [00:00, ?it/s]


358it [00:00, 3578.91it/s]


705it [00:00, 3506.69it/s]


1001it [00:00, 3536.31it/s]

9907it [48:03,  3.44it/s]


0it [00:00, ?it/s]


361it [00:00, 3609.21it/s]


713it [00:00, 3564.03it/s]


1001it [00:00, 3536.31it/s]

9908it [48:04,  3.44it/s]


0it [00:00, ?it/s]


357it [00:00, 3533.90it/s]


704it [00:00, 3519.06it/s]


1001it [00:00, 3487.03it/s]

9909it [48:04,  3.44it/s]


0it [00:00, ?it/s]


336it [00:00, 3326.00it/s]


686it [00:00, 3412.16it/s]


1001it [00:00, 3462.89it/s]

9910it [48:04,  3.44it/s]


0it [00:00, ?it/s]


336it [00:00, 3326.01it/s]


679it [00:00, 3377.36it/s]


1001it [00:00, 3392.36it/s]

9911it [48:04,  3.44it/s]


0it [00:00, ?it/s]


337it [00:00, 3369.30it/s]


688it [00:00, 3422.14it/s]


1001it [00:00, 3462.91it/s]

9912it [48:05,  3.44it/s]


0it [00:00, ?it/s]


345it [00:00, 3415.09it/s]


697it [00:00, 3484.

9965it [48:20,  3.44it/s]


0it [00:00, ?it/s]


353it [00:00, 3494.19it/s]


709it [00:00, 3544.07it/s]


1001it [00:00, 3511.50it/s]

9966it [48:20,  3.44it/s]


0it [00:00, ?it/s]


354it [00:00, 3504.19it/s]


702it [00:00, 3491.76it/s]


1001it [00:00, 3487.03it/s]

9967it [48:21,  3.44it/s]


0it [00:00, ?it/s]


347it [00:00, 3434.90it/s]


711it [00:00, 3554.04it/s]


1001it [00:00, 3561.48it/s]

9968it [48:21,  3.44it/s]


0it [00:00, ?it/s]


342it [00:00, 3385.38it/s]


693it [00:00, 3447.00it/s]


1001it [00:00, 3487.01it/s]

9969it [48:21,  3.44it/s]


0it [00:00, ?it/s]


354it [00:00, 3504.17it/s]


705it [00:00, 3506.66it/s]


1001it [00:00, 3536.31it/s]

9970it [48:22,  3.44it/s]


0it [00:00, ?it/s]


356it [00:00, 3523.98it/s]


713it [00:00, 3541.66it/s]


1001it [00:00, 3483.72it/s]

9971it [48:22,  3.44it/s]


0it [00:00, ?it/s]


327it [00:00, 3236.87it/s]


647it [00:00, 3218.19it/s]


997it [00:00, 3311.55it/s]


1001it [00:00, 3281.23it/s]

9972it [48:22,  3.4

In [69]:
# for storing the ranks of the duplicates
duplicate_rank = []

for line in data_processed:
    # get the question and duplicate candidates
    question, *candidates = line
    # get the rankings for the candidates
    candidate_rankings = rank_duplicate_candidates(question, candidates, word_embeddings)
    # store the index of the correct duplicate in rankings
    for rank,(index,_) in enumerate(candidate_rankings):
        if index == 0:
            duplicate_rank.append(rank+1)
            break

In [73]:
# check scores for different k values
print("    K|    DCG Score   |    Hits Score")
for k in [1, 5, 10, 50, 100, 200, 500, 1000]:
    print("%5d|    %.4f      |    %.4f" % 
          (k, dcg_k(k, duplicate_rank), hits_k(k, duplicate_rank)))

    K|    DCG Score   |    Hits Score
    1|    0.2944      |    0.2944
    5|    0.3706      |    0.4374
   10|    0.3873      |    0.4890
   50|    0.4122      |    0.6009
  100|    0.4212      |    0.6562
  200|    0.4297      |    0.7172
  500|    0.4433      |    0.8304
 1000|    0.4610      |    1.0000
